In [1]:
import pandas as pd
from sklearn.metrics import silhouette_score, pairwise_distances
import numpy as np
import scipy.stats as st


In [2]:
def shuffle_row(row):
    shuffled_row = row.values.copy()
    np.random.shuffle(shuffled_row)
    return pd.Series(shuffled_row, index=row.index)

def shuffle_rows(df):
    columns_to_shuffle = df.columns[1:]
    df[columns_to_shuffle] = df[columns_to_shuffle].apply(shuffle_row, axis=1)
    return df

In [3]:
def get_geom_mean_expression(expression_df):
    """
    
    Function to take an expression dataframe from the microarrays and collapse it into the means of
    all replicate chips.
    """
    # C2 and S12 got removed during quality control
    x = [
        'Ll', 
        'Lm', 
        'Lh', 
        'S0', 
        'S3', 
        'S6', 
        'S9', 
        # 'S12', 
        'S15', 
        'S24', 
        'C0', 
        # 'C2', 
        'C4', 
        'C6', 
        'C8', 
        'C10', 
        'C12', 
        'C14', 
        'C16', 
        'C18']
    
    # cols = expression_df.columns[1:]
    # x = [c for c in x if c in cols]
    
    condition_expr_dict = {c.split("_")[0]: [] for c in expression_df.columns[1:]}
    
    for c in list(expression_df.columns)[1:]:
        
        cond = c.split('_')[0]
        if cond in condition_expr_dict.keys():
            expr_list = condition_expr_dict.get(cond, [])

            # Need to avoid true zeros
            expr_list.append(expression_df[c].values)
            condition_expr_dict[cond] = expr_list
        
    condition_mean_dict = {c: (st.mstats.gmean(np.array(condition_expr_dict[c]) + 1, 0) - 1) for c in condition_expr_dict.keys() if c in x}
    
    mean_expr_df = pd.DataFrame(condition_mean_dict)
    mean_expr_df['TTHERM_ID'] = expression_df['TTHERM_ID'].values
    cols = list(mean_expr_df.columns)
    reorder = cols[-1:] + cols[:-1]
    mean_expr_df = mean_expr_df[reorder]
    
    return mean_expr_df

def normalizer(array):
    """
    Normalizes the values of an array to range from zero to one
    """
    
    a = np.array(array)
    
    normalized = (array - np.min(array)) / (np.max(array) - np.min(array))
    
    return normalized

def normalize_expression_per_gene(expression_df):
    """
    Function to normalize all gene expression to range from zero to one.
    """
    if 'TTHERM_ID' in expression_df.columns:
        ttids = expression_df['TTHERM_ID'].values
        data = expression_df[list(expression_df.columns)[1:]]
        
        norm_expression_df = data.apply(lambda row: normalizer(row), axis=1)
        norm_expression_df['TTHERM_ID'] = ttids
        
        columns = norm_expression_df.columns.tolist()
        
        rearrangment = columns[-1:] + columns[:-1]
        
        norm_expression_df = norm_expression_df[rearrangment]
        
    else:
        norm_expression_df = expression_df.apply(lambda row: normalizer(row), axis=1)
    
    return norm_expression_df

In [4]:
# Load your gene expression data
gene_data = pd.read_csv('../microarray_probe_alignment_and_filtering/allgood_filt_agg_tidy_2021aligned_qc_rma_expression_full.csv')  # Assuming the first column is the index
gene_data = gene_data.rename(columns={'Unnamed: 0': 'TTHERM_ID'})

In [5]:
gene_data.head()

,TTHERM_ID,Ll_GSM283687,Ll_GSM284355,Ll_GSM284362,Lm_GSM283690,Lm_GSM284357,Lm_GSM284363,Lh_GSM283691,Lh_GSM284360,Lh_GSM284364,...,C12_GSM656237,C14_GSM285580,C14_GSM285593,C14_GSM656238,C16_GSM285582,C16_GSM285595,C16_GSM656239,C18_GSM285583,C18_GSM285596,C18_GSM656240
0,TTHERM_000000042,6.928782,7.264201,6.934214,6.732989,6.970612,7.150978,6.126826,6.868968,6.641119,...,6.450318,8.049750,7.788162,7.052154,6.517742,6.918501,6.048861,7.041619,6.757932,5.817246
1,TTHERM_000000045,9.633489,9.977124,10.027529,9.720665,9.605762,10.225542,10.279608,10.459966,10.693337,...,11.130466,11.207738,11.009172,10.615417,11.038938,11.009222,10.216348,11.099187,11.172276,10.561021
2,TTHERM_00000010,5.066343,4.767264,5.010981,6.139047,4.619361,4.751761,5.818550,5.342529,5.483750,...,6.314438,7.423571,7.507645,7.417087,7.147801,7.747930,7.093641,7.672685,7.511290,6.890117
3,TTHERM_00000020,4.696881,4.638401,4.956299,6.942556,5.101252,4.730307,8.457690,4.526411,4.908300,...,5.250233,4.974993,5.747498,5.252167,5.210531,7.083187,5.252222,5.037613,5.495281,5.013987
4,TTHERM_00000030,4.654278,4.537105,4.928739,5.063991,4.584168,4.911880,5.935311,4.519470,4.757861,...,4.651688,4.920573,4.636333,4.883712,4.779395,4.744335,4.513140,4.838428,4.961475,4.653340


In [6]:
# gene_data = shuffle_rows(gene_data)

In [7]:
gene_data.head()

,TTHERM_ID,Ll_GSM283687,Ll_GSM284355,Ll_GSM284362,Lm_GSM283690,Lm_GSM284357,Lm_GSM284363,Lh_GSM283691,Lh_GSM284360,Lh_GSM284364,...,C12_GSM656237,C14_GSM285580,C14_GSM285593,C14_GSM656238,C16_GSM285582,C16_GSM285595,C16_GSM656239,C18_GSM285583,C18_GSM285596,C18_GSM656240
0,TTHERM_000000042,6.928782,7.264201,6.934214,6.732989,6.970612,7.150978,6.126826,6.868968,6.641119,...,6.450318,8.049750,7.788162,7.052154,6.517742,6.918501,6.048861,7.041619,6.757932,5.817246
1,TTHERM_000000045,9.633489,9.977124,10.027529,9.720665,9.605762,10.225542,10.279608,10.459966,10.693337,...,11.130466,11.207738,11.009172,10.615417,11.038938,11.009222,10.216348,11.099187,11.172276,10.561021
2,TTHERM_00000010,5.066343,4.767264,5.010981,6.139047,4.619361,4.751761,5.818550,5.342529,5.483750,...,6.314438,7.423571,7.507645,7.417087,7.147801,7.747930,7.093641,7.672685,7.511290,6.890117
3,TTHERM_00000020,4.696881,4.638401,4.956299,6.942556,5.101252,4.730307,8.457690,4.526411,4.908300,...,5.250233,4.974993,5.747498,5.252167,5.210531,7.083187,5.252222,5.037613,5.495281,5.013987
4,TTHERM_00000030,4.654278,4.537105,4.928739,5.063991,4.584168,4.911880,5.935311,4.519470,4.757861,...,4.651688,4.920573,4.636333,4.883712,4.779395,4.744335,4.513140,4.838428,4.961475,4.653340


In [8]:
gene_data = normalize_expression_per_gene(gene_data)

In [9]:
gene_data.head()

,TTHERM_ID,Ll_GSM283687,Ll_GSM284355,Ll_GSM284362,Lm_GSM283690,Lm_GSM284357,Lm_GSM284363,Lh_GSM283691,Lh_GSM284360,Lh_GSM284364,...,C12_GSM656237,C14_GSM285580,C14_GSM285593,C14_GSM656238,C16_GSM285582,C16_GSM285595,C16_GSM656239,C18_GSM285583,C18_GSM285596,C18_GSM656240
0,TTHERM_000000042,0.604560,0.712935,0.606315,0.541298,0.618075,0.676352,0.345444,0.585234,0.511614,...,0.449966,0.966750,0.882229,0.644422,0.471751,0.601238,0.320253,0.641018,0.549357,0.245417
1,TTHERM_000000045,0.015138,0.202748,0.230267,0.062732,0.000000,0.338374,0.367892,0.466359,0.593770,...,0.832424,0.874612,0.766203,0.551230,0.782454,0.766230,0.333354,0.815348,0.855251,0.521531
2,TTHERM_00000010,0.071967,0.023813,0.063054,0.244680,0.000000,0.021317,0.193078,0.116435,0.139173,...,0.272919,0.451498,0.465034,0.450454,0.407097,0.503722,0.398377,0.491607,0.465621,0.365608
3,TTHERM_00000020,0.049259,0.036473,0.105976,0.540233,0.137667,0.056567,0.871488,0.011989,0.095481,...,0.170239,0.110063,0.278956,0.170661,0.161559,0.570979,0.170673,0.123753,0.223814,0.118588
4,TTHERM_00000030,0.027748,0.004712,0.081708,0.108299,0.013964,0.078393,0.279603,0.001245,0.048113,...,0.027239,0.080103,0.024220,0.072855,0.052347,0.045454,0.000000,0.063952,0.088144,0.027564


In [20]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

encoding_dim = 128
epochs = 50
batch_size = 64


# Assuming df is your DataFrame
labels = gene_data.iloc[:, 0]
data = gene_data.iloc[:, 1:]

manhattan_distance_matrix = pairwise_distances(data, metric='manhattan', n_jobs=-1)

In [15]:
# Create a simple autoencoder model
input_layer = Input(shape=(data.shape[1],))
encoded = Dense(encoding_dim, activation='relu')(input_layer)
decoded = Dense(data.shape[1], activation='sigmoid')(encoded)

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Train the model with your unlabeled data
autoencoder.fit(data, data, epochs=epochs, batch_size=batch_size)

Epoch 1/50
318/318 [==============================] - 0s 674us/step - loss: 0.5898
Epoch 2/50
318/318 [==============================] - 0s 737us/step - loss: 0.5419
Epoch 3/50
318/318 [==============================] - 0s 735us/step - loss: 0.5352
Epoch 4/50
318/318 [==============================] - 0s 758us/step - loss: 0.5324
Epoch 5/50
318/318 [==============================] - 0s 679us/step - loss: 0.5309
Epoch 6/50
318/318 [==============================] - 0s 703us/step - loss: 0.5299
Epoch 7/50
318/318 [==============================] - 0s 692us/step - loss: 0.5292
Epoch 8/50
318/318 [==============================] - 0s 677us/step - loss: 0.5287
Epoch 9/50
318/318 [==============================] - 0s 733us/step - loss: 0.5284
Epoch 10/50
318/318 [==============================] - 0s 659us/step - loss: 0.5282
Epoch 11/50
318/318 [==============================] - 0s 662us/step - loss: 0.5280
Epoch 12/50
318/318 [==============================] - 0s 663us/step - loss: 0.5279
E

In [21]:
encoded_data = autoencoder.predict(data)

636/636 [==============================] - 0s 384us/step


In [22]:
from sklearn.cluster import KMeans

# Apply K-means clustering
num_clusters = 200
kmeans = KMeans(n_clusters=num_clusters)
clusters = kmeans.fit_predict(encoded_data)

/Users/michaelbertagna/anaconda3/envs/debug_tgne.env/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [23]:
silhouette_score(manhattan_distance_matrix, clusters, metric='precomputed')

0.051833360829634016